In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
# Configuration de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-rPBnsXi 78BbewkjRBfeT3BlbkFJoLeEVAbOl0kiGIZTWPKAtav6nWikZPSewB3oeo055nFcLW9rnMorepgfeAZNQQz5FUUwrtmQsA"


# Création du modèle LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [4]:
complexity_based_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate Complexity-based Prompting for complex reasoning tasks.
    
    Complexity-based Prompting is an advanced form of Chain-of-Thought (CoT) that focuses on complex examples and prioritizes more elaborate reasoning chains. 
    
    This technique significantly improves performance on mathematical and complex reasoning tasks by:
    1. Selecting and analyzing complex examples that require multiple reasoning steps
    2. Generating thorough reasoning chains that explore the problem space comprehensively
    3. Prioritizing more detailed explanations, under the premise that longer reasoning indicates higher quality answers
    
    The Complexity-based Prompting process includes the following steps:
    1. Problem complexity assessment: Evaluate the complexity of the problem based on length, required reasoning steps, and conceptual depth.
    2. Detailed reasoning chain: Develop an extensive, step-by-step reasoning process that thoroughly addresses all components of the problem.
    3. Alternative pathway consideration: Consider multiple approaches or solution paths to verify the answer.
    4. Final answer with justification: Provide a final answer with clear justification based on the most reliable reasoning chain.
    
    Please structure your response in the following format:
    
    Step 1: [Problem complexity assessment]
    Step 2: [Detailed reasoning chain]
    Step 3: [Alternative pathway consideration]
    Step 4: [Final answer with justification]
    
    Example:
    Question: "A delivery company charges $10 for the first kg and $7 for each additional kg. If a package costs $80 to ship, how many kg does it weigh?"
    
    Step 1: Problem complexity assessment:
    This is a multi-step algebraic word problem that requires:
    - Understanding the pricing structure (different rates for first vs. additional kg)
    - Setting up and solving an equation with variables
    - Checking if the solution makes sense in the context
    This problem has moderate complexity requiring at least 3-4 distinct reasoning steps.
    
    Step 2: Detailed reasoning chain:
    Let me denote the total weight as x kg.
    
    Given information:
    - First kg costs $10
    - Each additional kg costs $7
    - Total shipping cost is $80
    
    Let's set up an equation:
    - For x kg, we have 1 kg at $10 and (x-1) kg at $7 each
    - This gives us: 10 + 7(x-1) = 80
    
    Let's solve this equation:
    - 10 + 7x - 7 = 80
    - 7x + 3 = 80
    - 7x = 77
    - x = 77/7 = 11
    
    So according to this calculation, the package weighs 11 kg.
    
    Step 3: Alternative pathway consideration:
    Let me verify this result by calculating the cost for an 11 kg package:
    - First kg: $10
    - Additional 10 kg: 10 × $7 = $70
    - Total: $10 + $70 = $80
    
    This matches our given cost of $80.
    
    Let me also consider a different approach:
    - If the package weighed x kg, then the cost would be: 10 + 7(x-1)
    - We can rewrite this as: 10 + 7x - 7 = 3 + 7x
    - Setting this equal to $80: 3 + 7x = 80
    - Subtracting 3: 7x = 77
    - Dividing by 7: x = 11
    
    Both approaches yield the same result, confirming our answer.
    
    Step 4: The package weighs 11 kg. This answer is justified because:
    - The mathematical equation 10 + 7(x-1) = 80 yields x = 11
    - Verification shows that an 11 kg package would indeed cost $80 ($10 for the first kg plus $70 for the remaining 10 kg)
    - An alternative solution approach confirms the same result
    """),
    ("human", "{question}"),
    ("assistant", """
    Step 1: Problem complexity assessment:
    [Evaluate the complexity of the problem by identifying:
    - The number of variables, constraints, or relationships involved
    - The number of distinct reasoning steps required
    - The conceptual knowledge needed to solve the problem
    - Any potential subtleties or edge cases that need consideration]
    
    Step 2: Detailed reasoning chain:
    [Provide an extensive, step-by-step reasoning process that addresses all aspects of the problem:
    1. [Clear statement of the approach and initial setup]
    2. [Systematic development of the solution with careful attention to each step]
    3. [Explicit calculations or logical deductions shown in detail]
    4. [Continued detailed reasoning until reaching a provisional answer]
    ...]
    
    Step 3: Alternative pathway consideration:
    [Explore at least one alternative approach or verification method:
    - [Verification of the answer by working backwards]
    - [Alternative solution method that approaches the problem differently]
    - [Discussion of how the different approaches converge on the same answer or why one approach is more reliable]]
    
    Step 4: [Provide a clear, precise answer to the original question, with explicit justification based on the most reliable reasoning chain and verification process]
    """)
])

In [5]:
# Fonction pour générer le prompt ThoT (node)
def generate_thot_node(state):
    question = state['messages'][-1].content  # Récupère la dernière question
    prompt_value = complexity_based_prompt_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    return {"messages": [response]}  # Ajoute la réponse comme un message

# Définition de l'état
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Création du graphe
graph_builder = StateGraph(State)
graph_builder.add_node("generate_thot", generate_thot_node)

# Configuration du graphe
graph_builder.set_entry_point("generate_thot")
graph_builder.add_edge("generate_thot", END)
graph = graph_builder.compile()

# Exemple d'utilisation
inputs = {"messages": [HumanMessage(content="Design an algorithm to find the kth smallest element in an unsorted array of n distinct integers in expected O(n) time complexity. Explain your approach, analyze its time and space complexity, and demonstrate how it works on the array [9, 3, 2, 7, 5, 1, 8, 6, 4] for k=4.")]}

# Affiche la question posée au début
print("Question posée:", inputs["messages"][0].content)
print("\nRéponse:")

for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_thot":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)

Question posée: Design an algorithm to find the kth smallest element in an unsorted array of n distinct integers in expected O(n) time complexity. Explain your approach, analyze its time and space complexity, and demonstrate how it works on the array [9, 3, 2, 7, 5, 1, 8, 6, 4] for k=4.

Réponse:
Step 1: Problem complexity assessment:
The problem of finding the kth smallest element in an unsorted array of n distinct integers is a selection problem that can be solved efficiently. The complexity arises from the need to partition the array and recursively narrow down the search space. The expected time complexity is O(n) due to the use of the Quickselect algorithm, which is based on the partitioning method of QuickSort. The problem requires understanding of:
- Array manipulation
- Partitioning logic
- Recursive algorithms
This problem has moderate complexity, requiring several reasoning steps to implement and analyze the algorithm.

Step 2: Detailed reasoning chain:
To find the kth smalle